# Introduction

Analysis of a set of simulation outputs that are the result of a parameter sweep.

In [ ]:
# Paste output directories below (e.g use ctrl+c in Nautilus file manager)
outputs_clipboard = """

"""
output_dirs = outputs_clipboard.strip().split('\n')

# Firing Rates

# PSD

## Heatmap

## Bursting Power

# Spectrogram

## Peak Power

Note that this might give a different picture than comparing simulations with PSD alone: a simulation with periodic synchronization may have higher transient peaks, whereas the PSD (average over time) is lower.

# Synchronization